In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
import numpy as np
import os


In [7]:

# Define paths
data_dir = "waste dataset"  # Update to match your dataset structure
img_size = (224, 224)
batch_size = 32

# Data Preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=40,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Load Pretrained Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freeze first 100 layers
    layer.trainable = False

# Add Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

# Compile Model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


Found 4697 images belonging to 6 classes.
Found 1171 images belonging to 6 classes.


In [8]:

# Train Model
epochs = 30
model.fit(train_generator, validation_data=val_generator, epochs=epochs)

# Save Model
model.save("waste_classifier.h5")

print("Model training complete and saved as waste_classifier.h5")


Epoch 1/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.4429 - loss: 1.6827 - val_accuracy: 0.4791 - val_loss: 1.9763
Epoch 2/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 139s 948ms/step - accuracy: 0.7234 - loss: 0.8175 - val_accuracy: 0.5278 - val_loss: 1.6932
Epoch 3/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 142s 968ms/step - accuracy: 0.7882 - loss: 0.6593 - val_accuracy: 0.5628 - val_loss: 1.6246
Epoch 4/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.7994 - loss: 0.5690 - val_accuracy: 0.6285 - val_loss: 1.4066
Epoch 5/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 123s 841ms/step - accuracy: 0.8445 - loss: 0.4745 - val_accuracy: 0.6217 - val_loss: 1.5838
Epoch 6/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 123s 840ms/step - accuracy: 0.8548 - loss: 0.4242 - val_accuracy: 0.6772 - val_loss: 1.4201
Epoch 7/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 119s 808ms/step - accuracy: 0.8731 - loss: 0.3599 - val_accuracy: 0.6678 - val_loss: 1.3572
Epoch 8/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 123s 838ms/step - accuracy: 0.8705 - loss:

Model training complete and saved as waste_classifier.h5


# testing

In [9]:

# Class Labels
class_labels = ['e-waste', 'glass', 'metal', 'organic', 'paper', 'plastic']

# Function to Predict Image Class
def predict_image(image_path):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    predictions = model.predict(img_array)
    predicted_class = class_labels[np.argmax(predictions)]
    confidence = np.max(predictions)
    
    return predicted_class, confidence


In [13]:

# Example Usage
image_path = "waste dataset\\paper\\_105130812_img_1867.jpg"  # Change this to your image path
predicted_class, confidence = predict_image(image_path)
print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 896ms/step
Predicted Class: paper, Confidence: 0.57
